# Itertools

Prima di approfondire il modulo `itertools`, che implementa una serie di funzioni / "building blocks" per gestire in modo succinto ed efficace gli iteratori, ovvero tipi di dati che possono essere usati in un ciclo for, conviene approfondire ancora meglio il concetto di **iterator**.

L'iterator più comune (ma ce ne sono molti altri) in Python è la lista `list`, sulla quale è possibile appunto iterare.

In [1]:
colors = ['red', 'orange', 'yellow', 'green']
for each in colors:
    print(each)

red
orange
yellow
green


## Iteratori

E' possibile creare i propri oggetti `iterable`, da usarsi nei *loops* e per le *list comprehensions*.

Basta implementare i metodi *magici* necessari:

- `__iter__` che ritorna l'oggetto iterable, di solito self 
- `__next__` che definisce i valori dell'iteratore

Per esempio, vogliamo un iteratore che ritorni i numeri in ordine ad eccezione dei multipli dei numeri passati al costrutture.

Per essere sicuri di farlo terminare comunque, rilanceremo l'eccezione `StopIteration` dopo aver raggiunto i 200.

*Questo è un ottimo esempio di un'eccezione piuttosto comune in Python: gestire un evento che non sia inaspettato ma che richieda una fine; for loops e list comprehensions attendono `StopIteration` exception come segnale di stop.*

In [2]:
class NonFattoreIterable(object):
    def __init__(self, *args):
        # List of number for whic avoid multiples
        self.evita_multipli_di = args 
        self.x = 0
        
    def __next__(self):
        self.x += 1 
        while True: 
            if self.x > 200: 
                raise StopIteration 
            for y in self.evita_multipli_di: 
                if self.x % y == 0: 
                    self.x += 1 
                    break 
            else: 
                return self.x 
    
    def __iter__(self): 
        return self

In [3]:
es = NonFattoreIterable(3, 5)

In [4]:
[x for x in es]

[1,
 2,
 4,
 7,
 8,
 11,
 13,
 14,
 16,
 17,
 19,
 22,
 23,
 26,
 28,
 29,
 31,
 32,
 34,
 37,
 38,
 41,
 43,
 44,
 46,
 47,
 49,
 52,
 53,
 56,
 58,
 59,
 61,
 62,
 64,
 67,
 68,
 71,
 73,
 74,
 76,
 77,
 79,
 82,
 83,
 86,
 88,
 89,
 91,
 92,
 94,
 97,
 98,
 101,
 103,
 104,
 106,
 107,
 109,
 112,
 113,
 116,
 118,
 119,
 121,
 122,
 124,
 127,
 128,
 131,
 133,
 134,
 136,
 137,
 139,
 142,
 143,
 146,
 148,
 149,
 151,
 152,
 154,
 157,
 158,
 161,
 163,
 164,
 166,
 167,
 169,
 172,
 173,
 176,
 178,
 179,
 181,
 182,
 184,
 187,
 188,
 191,
 193,
 194,
 196,
 197,
 199]

In [5]:
quasi_primi = NonFattoreIterable(2, 3, 5, 7, 11, 13, 17, 19) 

In [6]:
[x for x in quasi_primi]

[1,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199]

In [7]:
somma_parziale = 0
for x in quasi_primi: 
    print(x)
    somma_parziale += x
somma_parziale # E' zero perchè l'iterabile è già stato "consumato" (da verificare!)

0

In [8]:
quasi_primi = NonFattoreIterable(2, 3, 5, 7, 11, 13, 17, 19) 
print(sum(quasi_primi))

4151


In [9]:
quasi_primi = NonFattoreIterable(2, 3, 5, 7, 11, 13, 17, 19)
print(max(quasi_primi))

199


Sembra inutile la presenza del metodo `__iter__`, ma ci sono casi in cui l'iterator di un oggetto non è lo stesso oggetto (occuparsi di questi casi ci farebbe però andare fuori tema).

### getItem

C'è un altro metodo che si può implementare un iterator personalizzato: il metodo `__getitem__`.
Questo permette la notazione parentesi quadra [] per ottenere il dato fuori dall'oggetto.

Comunque dobbiamo ricordarci di rilanciare `StopIteration` per farlo funzionare bene nei for e nelle list comprehensions.

In [10]:
class Quadrati(object): 
    def __init__(self, limite=200): 
        self.limite = limite 
        self.x = 0 

    def __next__(self): 
        self.x += 1 
        if self.x > self.limite: 
            raise StopIteration 
        return (self.x-1)**2 

    def __getitem__(self, idx): 
        # initialize counter to 0 
        self.x = 0 
        if not isinstance(idx, int): 
            raise Exception("Solo indici integer sono accettati!") 
        while self.x < idx: 
            self.__next__() 
        return self.x**2
    
    def __iter__(self): 
        return self

In [11]:
miei_quadrati = Quadrati(limite=20) 
[x for x in miei_quadrati]

[0,
 1,
 4,
 9,
 16,
 25,
 36,
 49,
 64,
 81,
 100,
 121,
 144,
 169,
 196,
 225,
 256,
 289,
 324,
 361]

In [12]:
miei_quadrati[5]

25

In [13]:
miei_quadrati[25]

StopIteration: 


Nell'esempio precedente abbiamo creato un iterator che ritorna quadrati di numeri. 

Nel metodo `__next__` si itera il nostro contatore ( `self.x` ) ritornando il suo quadrato finchè il contatore non supera il limte predefinito ( `self.limit`). 

Il while dell'esempio precedente (NonFattoreIterable) era specifico per quel caso d'uso.

Nel metodo `__next__` non è solitamente necessario implementare loop perchè normalmente viene richiamato all'interno di un loop sull'iterator.

Nell'esempio che segue implementiamo il metodo `__getitem__`, che ci permette di reperire un valore dell'iteratore ad una certa posizione di indice. 

Questo semplicemente chiama l'iteratore usando `self.__next__` finchè non arriva all'indice desiderato e restituisce il valore.

### Vantaggi degli iterator personalizzati

Perchè usare *iterator* personalizzati?

- Codice pulito
- Possibilità di lavorare con sequenze infinite
- Possibilità di usare funzioni built-in (come sum) che lavorano con iterable
- Possibilità di salvare memoria (es. range )

## Built-ins "itertools"

Uno dei tools sugli iterabili, fornito di base (built-in) è 

### zip()

che prende un numero qualsiasi di iterabili e ritorna un'iteratore di tuple con gli elementi corrispondenti:

In [14]:
list(zip([1, 2, 3], ['a', 'b', 'c']))

[(1, 'a'), (2, 'b'), (3, 'c')]

Un'altro operatore built-in sugli iteratori è

### map()  

che, nella sua forma più semplice, applica una funzione per ciascun elemento di un iterabile.

In [15]:
list(map(len, ['abc', 'de', 'fghi']))

[3, 2, 4]

## Il modulo itertools

Una collezione di *funzioni*

In [16]:
import itertools
import operator #Utile per alcuni degli esempi

### accumulate()

`itertools.accumulate(iterable[, func])`

Questa funziona crea un iteratore che ritorna il risultato di una funzione (passata come argomento) applicata su un iterabile e restituisce il risultato "accumulato".

I risultati sono contenuti essi stessi in un iterable.

Gli esempi possono aiutare la comprensione:

In [17]:
data = [1, 2, 3, 4, 5]
result = itertools.accumulate(data, operator.mul)
for each in result:
    print(each)

1
2
6
24
120


In [18]:
operator.mul(1, 2) # Prende due numeri e li moltiplica

2

In [19]:
data = [5, 2, 6, 4, 5, 9, 1]
result = itertools.accumulate(data, max)
for each in result:
    print(each)

5
5
6
6
6
9
9


In [20]:
max(5, 2) # Restituisce il massimo tra i due

5

Il passaggio di una funzione è opzionale, per default viene usata la somma.

In [21]:
data = [5, 2, 6, 4, 5, 9, 1]
result = itertools.accumulate(data)
for each in result:
    print(each)

5
7
13
17
22
31
32


### combinations()

`itertools.combinations(iterable, r)`

Dato un iterable e un intero, crea tutte le combinazioni con r membri.

In [22]:
shapes = ['circle', 'triangle', 'square',]
result = itertools.combinations(shapes, 2)
for each in result:
    print(each)

('circle', 'triangle')
('circle', 'square')
('triangle', 'square')


### combinations_with_replacement()

`itertools.combinations_with_replacement(iterable, r)`

Come **combinations()**, ma ammette ripetizioni.


In [23]:
shapes = ['circle', 'triangle', 'square',]
result = itertools.combinations_with_replacement(shapes, 2)
for each in result:
    print(each)

('circle', 'circle')
('circle', 'triangle')
('circle', 'square')
('triangle', 'triangle')
('triangle', 'square')
('square', 'square')


### count()

`itertools.count(start=0, step=1)`

Crea iterator che ritorna valori partendo dal valore start, applicando lo step desiderato.

In [24]:
for i in itertools.count(10,3):
    print(i)
    if i > 20:
        break

10
13
16
19
22


### cycle()

`itertools.cycle(iterable)`

Cicla sull'iteratore all'infinito.

In [25]:
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'violet']
for color in itertools.cycle(colors):
    print(color)

red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue

violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
green
blue
violet
red
orange
yellow
gre

KeyboardInterrupt: 

### chain()

`itertools.chain(*iterables)`

Prende una serie di iterable e ritorna come unico.

In [26]:
colors = ['red', 'orange', 'yellow', 'green', 'blue']
shapes = ['circle', 'triangle', 'square', 'pentagon']
result = itertools.chain(colors, shapes)
for each in result:
    print(each)

red
orange
yellow
green
blue
circle
triangle
square
pentagon


### compress()

`itertools.compress(data, selectors)`

Filtra un iterable con un altro

In [27]:
shapes = ['circle', 'triangle', 'square', 'pentagon']
selections = [True, False, True, False]
result = itertools.compress(shapes, selections)
for each in result:
    print(each)

circle
square


### dropwhile()

`itertools.dropwhile(predicate, iterable)`

Crea un iterator che elimina elementi da iterable finchè il predicato è True, dopodichè ritorna tutto il restante

In [28]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1]
result = itertools.dropwhile(lambda x: x<5, data)
for each in result:
    print(each)

5
6
7
8
9
10
1


### filterfalse()

`itertools.filterfalse(predicate, iterable)`

Crea un iterator che filtra elementi da iterable restituendo solo quelli per i quali il predicato è False.

In [29]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = itertools.filterfalse(lambda x: x<5, data)
for each in result:
    print(each)

5
6
7
8
9
10


### groupby()

`itertools.groupby(iterable, key=None)`

Semplicemente raggruppa, ma non è così semplice da spiegare...

In [30]:
robots = [{
    'name': 'blaster',
    'faction': 'autobot'
}, {
    'name': 'galvatron',
    'faction': 'decepticon'
}, {
    'name': 'jazz',
    'faction': 'autobot'
}, {
    'name': 'metroplex',
    'faction': 'autobot'
}, {
    'name': 'megatron',
    'faction': 'decepticon'
}, {
    'name': 'starcream',
    'faction': 'decepticon'
}]
for key, group in itertools.groupby(robots, key=lambda x: x['faction']):
    print(key)
    print(list(group))

autobot
[{'name': 'blaster', 'faction': 'autobot'}]
decepticon
[{'name': 'galvatron', 'faction': 'decepticon'}]
autobot
[{'name': 'jazz', 'faction': 'autobot'}, {'name': 'metroplex', 'faction': 'autobot'}]
decepticon
[{'name': 'megatron', 'faction': 'decepticon'}, {'name': 'starcream', 'faction': 'decepticon'}]


### islice()

`itertools.islice(iterable, start, stop[, step])`

Molto simile a **slices**, permette di tagliare una fetta di un iterable.

In [31]:
colors = ['red', 'orange', 'yellow', 'green', 'blue',]
few_colors = itertools.islice(colors, 2)
for each in few_colors:
    print(each)

red
orange


### permutations()

`itertools.permutations(iterable, r=None)`

Restituisce un iterabile con tutte le tuple combinazione

In [32]:
alpha_data = ['a', 'b', 'c']
result = itertools.permutations(alpha_data)
for each in result:
    print(each)

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')


### product()

`itertools.product(iterable1, iterable2, ...)`

Crea un iterable di tuple prodotto cartesiano da una serie di iterabili

In [33]:
num_data = [1, 2, 3]
alpha_data = ['a', 'b', 'c']
result = itertools.product(num_data, alpha_data)
for each in result:
    print(each)

(1, 'a')
(1, 'b')
(1, 'c')
(2, 'a')
(2, 'b')
(2, 'c')
(3, 'a')
(3, 'b')
(3, 'c')


### repeat()

`itertools.repeat(object[, times])`

Ripete un oggetto all'infinito a meno che venga specificato quante (times) volte

In [34]:
for i in itertools.repeat("spam",10):
    print(i)

spam
spam
spam
spam
spam
spam
spam
spam
spam
spam


### starmap()

`itertools.starmap(function, iterable)`

Crea un iterator che esegue la funzione usando gli argomenti ottenuti da iterable

In [35]:
data = [(2, 6), (8, 4), (7, 3)]
result = itertools.starmap(operator.mul, data)
for each in result:
    print(each)

12
32
21


### takewhile()

`itertools.takewhile(predicate, iterable)`

Opposto di `dropwhile()`, crea un iterator e restutisce elementi da iterable finchè il predicato è True.

In [36]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1]
result = itertools.takewhile(lambda x: x<5, data)
for each in result:
    print(each)

1
2
3
4


### tee()

`itertools.tee(iterable, n=2)`

Restituisce n iterators indipendenti da un singolo iterable; default è 2.

In [37]:
colors = ['red', 'orange', 'yellow', 'green', 'blue']
alpha_colors, beta_colors = itertools.tee(colors)
for each in alpha_colors:
    print(each)
print('..')
for each in beta_colors:
     print(each)

red
orange
yellow
green
blue
..
red
orange
yellow
green
blue


### zip_longest()

`itertools.zip_longest(*iterables, fillvalue=None)`

Crea un iterator che aggrega elementi per ciascun iterable. 
Se hanno lunghezza differente, i valori mancanti sono riempiti con fillvalue fino all'iterabile più lungo.

In [38]:
colors = ['red', 'orange', 'yellow', 'green', 'blue',]
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,]
for each in itertools.zip_longest(colors, data, fillvalue=None):
    print(each)

('red', 1)
('orange', 2)
('yellow', 3)
('green', 4)
('blue', 5)
(None, 6)
(None, 7)
(None, 8)
(None, 9)
(None, 10)


## Esempi

### Es. 1

Avete tre biglietti da 20 EUR, cinque da 10 EUR, due da 5 EUR e cinque monete da 1 EUR.

In quanti modi potete comporre 100 EUR ?

In [39]:
soldi = [20, 20, 20, 10, 10, 10, 10, 10, 5, 5, 1, 1, 1, 1, 1]

# Usiamo le combinazioni
fai_100 = []
for n in range(1, len(soldi) + 1):
    for comb in itertools.combinations(soldi, n):
        if sum(comb) == 100:
            fai_100.append(comb)

In [40]:
fai_100

[(20, 20, 20, 10, 10, 10, 10),
 (20, 20, 20, 10, 10, 10, 10),
 (20, 20, 20, 10, 10, 10, 10),
 (20, 20, 20, 10, 10, 10, 10),
 (20, 20, 20, 10, 10, 10, 10),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 10, 10, 10, 10, 10, 5, 5),
 (20, 20, 10, 10, 10, 10, 10, 5, 5),
 (20, 20, 10, 10, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),

Meglio rimuovere i duplicati

In [41]:
set(fai_100)

{(20, 20, 10, 10, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 10, 10, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
 (20, 20, 20, 10, 10, 10, 5, 5),
 (20, 20, 20, 10, 10, 10, 10)}

## Es. 2 - Variazione di 1

In quanti modi si possono comporre 100 EUR usando un numero qualsiasi di pezzi da 50, 20, 10, 5, e 1 EUR?

In [42]:
# Ci metterà un po' di tempo! 96560645 combinazioni
pezzi = [50, 20, 10, 5, 1]
fa_100 = []
for n in range(1, 101):
     for comb in itertools.combinations_with_replacement(pezzi, n):
        if sum(comb) == 100:
             fa_100.append(comb)

In [43]:
len(fa_100)

343

# Bibliografia

https://medium.com/@jasonrigden/a-guide-to-python-itertools-82e5a306cdf8

https://realpython.com/python-itertools/